In [41]:
from pyspark.sql import HiveContext
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, ArrayType, MapType

## DataFrame de vídeos em alta por país

#### Lendo todos os arquivos csv

In [120]:
df_paises = spark.read.csv('/datasets/youtube/*.csv', header=True)

#### Criando uma coluna para classificar os vídeos em alta por país, baseado no nome do arquivo

In [121]:
df_paises = df_paises.withColumn('country', F.regexp_extract(F.input_file_name(), r"/([^/]+)_youtube_trending_data\.csv$", 1))

In [122]:
df_paises.head(2)

[Row(video_id='s9FH4rDMvds', title='LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!', publishedAt='2020-08-11T22:21:49Z', channelId='UCGfBwrCoi9ZJjKiUK8MmJNw', channelTitle='Pietro Guedes', categoryId='22', trending_date='2020-08-12T00:00:00Z', tags='pietro|guedes|ingrid|ohara|pingrid|vlog|amigos|jully|molina|mansão|mansao|dos|youtubers|gkhouse', view_count='263835', likes='85095', dislikes='487', comment_count='4500', thumbnail_link='https://i.ytimg.com/vi/s9FH4rDMvds/default.jpg', comments_disabled='False', ratings_disabled='False', description='Salve rapaziada, neste vídeo me declarei pra ela e me surpreendi com a resposta! Espero que gostem! =)MINHAS REDES- Instagram: https://www.instagram.com/190pietro/?hl=pt-br- TikTok: https://www.tiktok.com/@190pietro?lang=pt_BR - Twitter: https://mobile.twitter.com/ttdo190►CANAIS DOS PARTICIPANTES E PLAYLIST TODOS VÍDEOS DA CASA:Playlist ➜ https://www.youtube.com/playlist?list=PLrT75fWIkg088N2Q_h1Hw5Lq-j9YWxAOCCanais: ➜ Gregory Kessey: https://

## DataFrame de categorias de vídeo

#### Lendo todos os arquivos json

In [124]:
df_category = spark.read.json('/datasets/youtube/*.json', multiLine=True)

#### Criando uma coluna para classificar as categorias por país, baseado no nome do arquivo

In [125]:
df_category = df_category.withColumn("country", F.regexp_extract(F.input_file_name(), r"/([^/]+)_category_id\.json$", 1))#F.input_file_name())

#### Parseando coluna json items

In [126]:
schema = ArrayType(StringType())

df_category = (df_category
               .withColumn('items', F.explode('items'))   
               .withColumn('item_etag', F.col("items").getItem('etag'))
               .withColumn('categoryId', F.col("items").getItem('id'))
               .withColumn('snippet', F.col("items").getItem('snippet'))
               .withColumn('snippet', F.col("items").getItem('snippet'))
               .withColumn('title', F.col("snippet").getItem('title'))
               .withColumn('assignable', F.col("snippet").getItem('assignable'))
               .withColumn('channelId', F.col("snippet").getItem('channelId'))
               .select([
                   F.col('kind'),
                   F.col('etag'),
                   F.col('item_etag'),
                   F.col('categoryId'), 
                   F.col('title'), 
                   F.col('assignable'), 
                   F.col('channelId'),
                   F.col('country')
               ])
              )         

In [127]:
df_category.head(2)

[Row(kind='youtube#videoCategoryListResponse', etag='HIrK3n45Uw2IYz9_U2-gK1OsXvo', item_etag='IfWa37JGcqZs-jZeAyFGkbeh6bc', categoryId='1', title='Film & Animation', assignable=True, channelId='UCBR8-60-B28hp2BmDPdntcQ', country='US'),
 Row(kind='youtube#videoCategoryListResponse', etag='HIrK3n45Uw2IYz9_U2-gK1OsXvo', item_etag='5XGylIs7zkjHh5940dsT5862m1Y', categoryId='2', title='Autos & Vehicles', assignable=True, channelId='UCBR8-60-B28hp2BmDPdntcQ', country='US')]

## DataFrame final de vídeos em alta e categoria

In [128]:
df = df_paises.join(df_category, on=['categoryId','country'])

In [129]:
df.head(3)

[Row(categoryId='22', country='BR', video_id='s9FH4rDMvds', title='LEVEI UM FORA? FINGI ESTAR APAIXONADO POR ELA!', publishedAt='2020-08-11T22:21:49Z', channelId='UCGfBwrCoi9ZJjKiUK8MmJNw', channelTitle='Pietro Guedes', trending_date='2020-08-12T00:00:00Z', tags='pietro|guedes|ingrid|ohara|pingrid|vlog|amigos|jully|molina|mansão|mansao|dos|youtubers|gkhouse', view_count='263835', likes='85095', dislikes='487', comment_count='4500', thumbnail_link='https://i.ytimg.com/vi/s9FH4rDMvds/default.jpg', comments_disabled='False', ratings_disabled='False', description='Salve rapaziada, neste vídeo me declarei pra ela e me surpreendi com a resposta! Espero que gostem! =)MINHAS REDES- Instagram: https://www.instagram.com/190pietro/?hl=pt-br- TikTok: https://www.tiktok.com/@190pietro?lang=pt_BR - Twitter: https://mobile.twitter.com/ttdo190►CANAIS DOS PARTICIPANTES E PLAYLIST TODOS VÍDEOS DA CASA:Playlist ➜ https://www.youtube.com/playlist?list=PLrT75fWIkg088N2Q_h1Hw5Lq-j9YWxAOCCanais: ➜ Gregory Ke